<a href="https://colab.research.google.com/github/surbhiagrawal22/MachineLearning-DeepLearning-codes/blob/master/Bank_customer_churning_Classification_task_using_deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

Reading the Dataset and analysing basic details about dataset

In [ ]:
pd.set_option("display.max_columns",None)

data_df=pd.read_csv("https://raw.githubusercontent.com/krishnaik06/Complete-Deep-Learning/master/ANN/Churn_Modelling.csv")

print(data_df.head())
print(data_df.tail(5))
print(data_df.sample(5))


In [ ]:
# checking for presence of any null values in dataset
print(data_df.info())
print(data_df.describe())

In [ ]:
encoder=OneHotEncoder()
new_geography= encoder.fit_transform(data_df[["Geography"]]).toarray()
new_gender=encoder.fit_transform(data_df[["Gender"]]).toarray()
data_df=data_df.join(pd.DataFrame(new_geography,columns=encoder.categories_)  )
print(data_df)
# print(new_geography)
# print(new_gender)

In [ ]:
print(data_df["Gender"].value_counts())
print(data_df["Geography"].value_counts())

print(data_df["Exited"].value_counts()) # as we see the value counts for exited, the dataset is imbalanced

In [ ]:
# defining the response variable and independent varaibles
X=data_df.drop(["Exited","RowNumber",  "CustomerId" ,  "Surname"],axis=1)
y=data_df["Exited"]
print(X.head())
print(y.head())

In [ ]:
# performing train test split
state=6
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.2,random_state=state,stratify=y)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(y_train.value_counts())
print(y_test.value_counts())